# Capstone Project
### Applied Data Science by IBM
### Michael Lynch
### April 12, 2020

## Table of contents
* [Introduction: Business Problem](#Introduction:-Business-Problem)
* [Data](#Data)
* [Methodology](#Methodology)
* [Analysis](#Analysis)
* [Results and Discussion](#Results-and-Discussion)
* [Conclusion](#Conclusion)

# Introduction: Business Problem

In this project we will analyze the spread of COVID-19 over time and space. We will also try to predict to increase in number of cases into the near future. This is important because it will help others see trends in the spread and will help myself learn python and complete the CourseEra Capstone course for IBM data science.

# Data

There is plentiful data for COVID-19 stats. Datasets can be found on GitHub and Kaggle.com. In this project we will utilze three datasets. The first dataset will be from covidtracking.com and it contains the data we will need to plot the rise in covid cases over time as well as predict the rise in cases into the near future. The second dataset is from John Hopkins' GitHub repository. This dataset contains covid-19 stats for each county in the USA which. The third dataset is from the NYTimes GitHub repository and is similar to the second dataset, except it doesn't come with the geospatial coordinates for each county. We will utilize a Python library called geocoder to get thsi information. Finally, we use the third dataset to make a heatmap with of covid cases that changes with each day so that we can visualize the spread of covid cases over time and space.

# Methodology

We will use the SVM machine learning algorithm to predict the increase in number of cases for each state in the USA. To find the best hyperparameters for the algorithm we will use a speacialized function in ski-kit-learn. We will see that the function did a good job in predicting the best hyperparameters by plotting the error with respect to one of the hyperparameters and see the error is lowest with the chosen hyperparameter. We will also use k-means clustering to find regions that are most concentrated with cases of COVID-19. We would expect regions high in population to be highest in cases of COVID-19.

# Analysis

### First, we import all nessesary libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import folium
from sklearn.cluster import KMeans
!pip install geocoder
import geocoder
import json
import time
import datetime
!pip install wget
import wget

from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Our first dataset is downloaded from https://covidtracking.com/. Note that the dataset is updated each day. In this first analysis we will plot and predict the number of cases in a specific state, then will will use for loops to do the same for all states.

### Try to open the dataset in your current folder if you already have it saved. If it's not there, download and use the one from the website.

In [ ]:
try:
    with open('daily.csv') as us_states_covid19_daily:
        df=pd.read_csv(us_states_covid19_daily)
except IOError: 
    url='https://covidtracking.com/api/v1/states/daily.csv'
    df=pd.read_csv(url)  

In [ ]:
df.head()

### Sort by state primarily and date secondarly

In [ ]:
df.sort_values(['state','date'],inplace=True,ascending=(True,False))
df.head()

### Make a list of all states. Note that this list actually contains territories to. We will make a revised list fo states later.

In [ ]:
states=df['state'].unique()

### Delete Columns that don't have useful data that we care about

In [ ]:
df=df.drop(['dateChecked','hash','pending','negative','deathIncrease','hospitalizedIncrease','negativeIncrease','positiveIncrease','totalTestResultsIncrease','fips','total','totalTestResults','posNeg'],axis=1)
df.head()

### The below cell splits the dataset into len(states) number of datasets, one for each state

In [ ]:
df_dict={}
for i in range(len(states)):
    df_dict[states[i]]=[]
    for j in range(len(df)):
        if df['state'].iloc[j]==states[i]:
            df_dict[states[i]].append(df.iloc[j])

In [ ]:
for i in range(len(df_dict)):
    df_dict[states[i]]=pd.DataFrame(df_dict[states[i]])

### Confirm that worked for a specific state

In [ ]:
df_dict['MN']

### Get a list of all the dates included in the dataset so that we can use it as our x-axis in some plots. Convert the dates to number of days since the first date. Also cast the data into a numpy array and reshape so that it is in the correct format for feeding into the SVM algorithm

In [ ]:
dates_mn=df_dict['MN']['date']
dates_mn=pd.to_datetime(dates_mn, format='%Y%m%d')
day_num_mn=[i for i in range(len(dates_mn))]
day_num_mn=np.array(day_num_mn).reshape(-1,1)
#mn_dates=pd.Index(df_dict['MN']['date'].unique())

In [ ]:
cases_mn=np.array(df_dict['MN']['positive'].tolist())
cases_mn=np.flip(cases_mn).reshape(-1,1)

In [ ]:
plt.plot(day_num_mn,cases_mn)
plt.title('Number of People Tested Positive for COVID-19 in MN')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.ylabel('Number of People')
plt.show()

### Define the number of days into the future we want to SVM algorithm to predict

In [ ]:
days_in_future = 10
future_forcast_mn = np.array([i for i in range(len(dates_mn)+days_in_future)]).reshape(-1, 1)

### Split data into training and testing sets

In [ ]:
X_train_mn, X_test_mn, y_train_mn, y_test_mn = train_test_split(day_num_mn, cases_mn, test_size=0.1, shuffle=False) 

### The following lines of code I altered from: https://www.kaggle.com/therealcyberlord/coronavirus-covid-19-visualization-prediction

In [ ]:
# use this to find the optimal parameters for SVR
#c = [0.01, 0.1, 1]
#gamma = [0.01,0.015,0.02] #I had to alter this line because my system would crash with larger values
#epsilon = [0.01, 0.1, 1]
#shrinking = [True, False]
#degree = [3, 4, 5, 6, 7]
#
#svm_grid = {'C': c, 'gamma' : gamma, 'epsilon': epsilon, 'shrinking' : shrinking, 'degree': degree}
#
#svm = SVR(kernel='poly')
#svm_search = RandomizedSearchCV(svm, svm_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=30, verbose=1)
#svm_search.fit(X_train_mn, np.ravel(y_train_mn))
#
#svm_search.best_params_

#svm_confirmed_mn = svm_search.best_estimator_

### After finding the best parameters I kept them so I didn't always have to re-run teh above cell

In [ ]:
#svm_confirmed_mn = svm_search.best_estimator_
svm_confirmed_mn = SVR(shrinking=False, kernel='poly',gamma=0.01, epsilon=0.1,degree=3, C=1)
svm_confirmed_mn.fit(X_train_mn, np.ravel(y_train_mn))
svm_pred_mn = svm_confirmed_mn.predict(future_forcast_mn)

### Get the Mean Absolute Error for degrees 1 thru 9 so that we can make a plot of that

In [ ]:
MAE=[]
for i in range(1,10):
    svm_confirmed_degree_test = SVR(shrinking=False, kernel='poly',gamma=0.01, epsilon=0.1,degree=i, C=1)
    svm_confirmed_degree_test.fit(X_train_mn, np.ravel(y_train_mn))
    svm_pred_degree_test = svm_confirmed_degree_test.predict(future_forcast_mn)
    svm_test_pred_degree_test = svm_confirmed_degree_test.predict(X_test_mn)
    MAE.append(mean_absolute_error(svm_test_pred_degree_test, y_test_mn))


### Make a plot of degree vs. error to confirm that are hyperparameters are well chosen.

In [ ]:
degree_num=list(range(1,10))
plt.plot(degree_num,MAE)
plt.title('Tuning Degree Parameter')
plt.ylabel('SVM Error')
plt.xlabel('Degree Number')
plt.show()

### Plot the predicted number of cases on top of the actual number of cases

In [ ]:
plt.plot(day_num_mn, cases_mn)
plt.plot(future_forcast_mn, svm_pred_mn, linestyle='dashed', color='purple')
plt.title('Predicted Increase in Number of Cases')
plt.legend(['Confirmed Cases','SVM Prediction'])
plt.xlabel('Day')
plt.ylabel('Number of Confirmed Cases')
plt.show()

### Get rid of the elements in the data that are not states. This will make our subplot below look better because we can shape it as 5x10, which we can do with 50 elements, but not 56.

In [ ]:
not_states=['AS','GU','DC','PR','MP','VI']
for i in range(len(not_states)):
    del df_dict[not_states[i]]
key_list=list(df_dict.keys())

### Below we make a list of all states and repeat the process we did above on MN for all states using a lot of for loops.

In [ ]:
states_list=[]
for i in range(len(states)):
    if states[i] not in not_states:
        states_list.append(states[i])

In [ ]:
dates=[]
for i in range(len(states_list)):
    dates.append(pd.Index(df_dict[states_list[i]]['date'].unique()))
day_num=[]
for i in range(len(dates)):
    day_num.append([j for j in range(len(dates[i]))])
    day_num[i]=np.array(day_num[i]).reshape(-1,1)

In [ ]:
day_num_length_list=[]
for i in range(len(day_num)):
    day_num_length_list.append(len(day_num[i]))
    
min_day_num=min(day_num_length_list)

In [ ]:
cases=[]
for i in range(len(df_dict)):
    cases.append(np.array(df_dict[states_list[i]]['positive'].tolist()))
    cases[i]=np.flip(cases[i]).reshape(-1,1)

In [ ]:
#cases_us=[]
#for day in list(df['date'].unique()):
#    cases_us.append(df[df['date']==day]['positive'].sum())
#cases_us=np.array(cases_us)
#cases_us=np.flip(cases_us).reshape(-1,1)


In [ ]:
days_in_future=10
future_forcast=[]
for i in range(len(dates)):
    future_forcast.append(np.array([j for j in range(len(dates[i])+days_in_future)]).reshape(-1, 1))

In [ ]:
svm_confirmed = svm_confirmed_mn

X_train=[0 for i in range(len(cases))]
X_test=[0 for i in range(len(cases))]
y_train=[0 for i in range(len(cases))]
y_test=[0 for i in range(len(cases))]
for i in range(len(cases)):
    X_train[i], X_test[i], y_train[i], y_test[i] = train_test_split(day_num[i], cases[i], test_size=0.1, shuffle=False)

In [ ]:
svm_pred=[0 for i in range(len(cases))]
for i in range(len(cases)):
        svm_confirmed.fit(X_train[i], np.ravel(y_train[i]))
        svm_pred[i] = svm_confirmed.predict(future_forcast[i])

### Make a plot of the number of cases vs time for each of the fifty states of the US.

In [ ]:
plt.figure(figsize=(8,8),dpi=100)
for i in range(50):
    plt.subplot(10,5,i+1)    
    plt.plot(day_num[i],cases[i])
    plt.title(key_list[i],loc='center',pad=-10,fontsize=10)
    plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
    plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected  
    direction='in',
    labelsize=5,
    pad=-20)

### Make a plot of the number of predicted cases for each state on top of the actual data for each state

In [ ]:
plt.figure(figsize=(8,8),dpi=100)
for i in range(50):
    plt.subplot(10,5,i+1)    
    plt.plot(day_num[i],cases[i])
    plt.plot(future_forcast[i], svm_pred[i], linestyle='dashed', color='purple')
    plt.title(states_list[i],loc='center',pad=-10,fontsize=10)
    plt.legend(['Actual','Predicted'],fontsize=3)
    plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
    plt.tick_params(
    axis='y',         
    which='both',      
    direction='in',
    labelsize=5,
    pad=-20)

### Now let's make dataframe with only the cumulative data from the most recent date of each state. That way we can see how the virus is spread over space.

In [ ]:
df_cum=df.sort_index()
df_cum.head()

### Retrieve only the first 56 rows because that is today's date and drop all unnessary columns

In [ ]:
df_cum=df_cum.iloc[0:56]

In [ ]:
#get rid of rows that are us territories, not states: AS, GU, DC, PR, MP, VI
drop_ind=[3,8,12,27,42,50]
df_cum.drop(index=drop_ind,inplace=True)

In [ ]:
df_cum.reset_index(drop=True,inplace=True)
#df_cum['positive'].astype('int',inplace=True)

### I had to create the following list with the states organized alphabetically by their two letter designation

In [ ]:
state_names=['Alaska','Alabama','Arkansas','Arizona','California','Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Indiana','Idaho', 'Illinois', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland','Maine','Michigan', 'Minnesota', 'Missouri','Mississippi', 'Montana', 'North Carolina', 'North Dakota','Nebraska', 'New Hampshire','New Jersey','New Mexico','Nevada','New York', 'Ohio','Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina','South Dakota', 'Tennessee', 'Texas', 'Utah','Virginia','Vermont', 'Washington', 'Wisconsin','West Virginia', 'Wyoming']

In [ ]:
df_cum['State_Names']=pd.DataFrame(state_names)

In [ ]:
df_cum=df_cum[['date','state','State_Names','positive','hospitalizedCurrently','inIcuCurrently','inIcuCumulative','onVentilatorCurrently','onVentilatorCumulative','recovered','death','hospitalized']]

### Let's view the whole dataset to confirm the two letter designation matches with the full state name

In [ ]:
df_cum

### Import a json file containing the state boundaries so we can use it for a choropleth map. If it's not in your current working directory download it from GitHub using the wget library. State boundaries json downloaded from https://github.com/PublicaMundi/MappingAPI/blob/master/data/geojson/us-states.json?short_path=1c1ebe5

In [ ]:
try:
    with open('us-states.json') as jsonfile:
        json_path=jsonfile.name
except IOError:
    #import wget
    us_states_json_url='https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json'
    us_states_json=wget.download(us_states_json_url)
    json_path=us_states_json

In [ ]:
us_map = folium.Map(location=[37,-96], zoom_start=4)

### We bin the data so that the colors on the chororpleth map are nicely spread

In [ ]:
bins = list(df_cum['positive'].quantile([0, 0.25, 0.5, 0.75, 1]))
bins

### Make a choropleth map of the number of cases in each state

In [ ]:
folium.Choropleth(
    geo_data=json_path,
    data=df_cum,
    columns=['State_Names', 'positive'],
    key_on='feature.properties.name',
    fill_color='YlGn', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='COVID-19',
    bins=bins,
    reset=True
).add_to(us_map)


# display map
us_map

## Dataset 2: Next we will import a database of all COVID-19 cases in each county of the US so that we can use it to to run k-means clustering to map the areas of the country that are dense in COVID-19 cases. Dataset from John Hopkins GitHub Repository: https://github.com/CSSEGISandData/COVID-19

### Similar code to what we did above with importing the first dataset

In [ ]:
try:
    with open('time_series_covid19_confirmed_US.csv') as timeseries:
        df_counties=pd.read_csv(timeseries)
except IOError:
    counties_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
    df_counties=pd.read_csv(counties_url)

In [ ]:
df_counties.head(10)

### Delete the columns we don't care about

In [ ]:
df_counties=df_counties.drop(['UID','iso2','iso3','code3','FIPS'],axis=1)
df_counties.head(10)

### Make a dataframe with just latitude, longitude, and number of cases

In [ ]:
most_recent_day='4/11/20'
df_counties_small=df_counties[['Lat','Long_',most_recent_day]]
df_counties_small.head()

### Setup our kmean parameters and run kmeans to get the centers where cases are the most dense

In [ ]:
kclusters=100
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(df_counties_small)
centers=kmeans.cluster_centers_

In [ ]:
lats=pd.Series(centers[:,0])
longs=pd.Series(centers[:,1])
weights=centers[:,2]

### Now we make the map of the cluster centers

In [ ]:
cluster_map=folium.Map(location=[37,-96], zoom_start=4)
for lat,long in zip(lats,longs):
    folium.CircleMarker(
            [lat,long],
            radius=5).add_to(cluster_map)

cluster_map

### The clusters seemed to be centered around areas that have large populations, which makes sense.

##  Dataset 3: Let's use another dataset which doesn't explictly have the latitudes and longitudes for each county. We can use geocoding to get the latitude and longitude for each county if we aren't explictly given it. The dataset we will use is found in the NY Times GitHub Repository here: https://github.com/nytimes/covid-19-data

### Import the data using similar code we did for the first and second datasets

In [ ]:
try:
    with open('us-counties.csv') as uscounties:
        df_counties2=pd.read_csv(uscounties)
except IOError:
    counties2_url='https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
    df_counties2=pd.read_csv(counties2_url)


In [ ]:
df_counties2.head()

### We make a column of the combined name of county and state. If the county name is unknown this causes problems with the geocoding API. So that we don't get that problem we don't include the county name there.

In [ ]:
for i in range(len(df_counties2)):
    if df_counties2['county'][i]!='Unknown':
        df_counties2.at[i,'county, state']=df_counties2['county'][i]+', '+df_counties2['state'][i]
    elif df_counties2['county'][i]=='Unknown':
        df_counties2.at[i,'county, state']=df_counties2['state'][i]
df_counties2.tail()

In [ ]:
county_state_list=list(df_counties2['county, state'].unique())
county_state_list
len(county_state_list)

## **The next few cells take a long time because they send many calls to the API. Be aware of the API usage restrictions, quotas, and limits. To bypass this I split the API calls up into 3 chunks and ran each chunk separatly. I saved the data after running each chunk. If you have problems running this part you can just download the file from my github, but I can't promise it will be up to date: https://github.com/mjlynch91/Coursera_Capstone/blob/master/county_lat_lng.txt

### You really have to be careful which cells you run the in next few cells depending on whether you are downloading the data, or just loading the data from you local disk.

In [ ]:
API_KEY='Put Your Api key here'
#g_total=[]
#for name in county_state_list[0:1000]:
#    g_current = geocoder.google(name,key=API_KEY)
#    g_total.append(g_current)

In [ ]:
#for name in county_state_list[1000:2000]:
#    g_current = geocoder.google(name,key=API_KEY)
#    g_total.append(g_current)

In [ ]:
#for name in county_state_list[2573:len(county_state_list)]:
#    g_current = geocoder.google(name,key=API_KEY)
#    g_total.append(g_current)

### So that we don't have to run the above cell everytime, I've saved the data to a file

In [ ]:
#g_dict={}
#g_dict['counties']=[]
#for i in range(len(g_total)):
 #   if type(g_total[i])==type(g_current):
  #      g_dict['counties'].append(g_total[i].json)
   # else:
    #    g_dict['counties'].append(g_total[i])

In [ ]:
#with open('county_lat_lng.txt', 'w') as outfile:
 #   json.dump(g_dict, outfile)

In [ ]:
#for i in range(len(g_total)):
 #   if type(g_total[i])!=dict:
  #      print(i,type(g_total[i]))

### Read the file from your local disk if you already have it.

In [ ]:
jsonpath='../input/county-lat-lng-json/county_lat_lng.json'
with open(jsonpath,"r") as json_file:
    g_dict = json.load(json_file)

In [ ]:
g_total=g_dict['counties']

### Now that we have the coordinates, we put them into the dataframe

In [ ]:
# g_current = geocoder.google('Minneasota',key=API_KEY)
g_current=0
lat_long=[]
for i in range(len(g_total)):
    if type(g_total[i])==type(g_current):
        lat_long.append([g_total[i].lat, g_total[i].lng])
    else:
        lat_long.append([g_total[i]['lat'], g_total[i]['lng']])
    

In [ ]:
#changing lat_long to a numpy array since it's easier to get columns and rows out
lat_long=np.array([[i[0] for i in lat_long],[i[1] for i in lat_long]])

In [ ]:
county_dict={}
for i in range(len(county_state_list)):
    county_dict.update({county_state_list[i]:tuple(lat_long[:,i])})

In [ ]:
lat_series=[]
long_series=[]
for i in range(len(df_counties2)):
    lat_series.append(county_dict[df_counties2['county, state'][i]][0])
    long_series.append(county_dict[df_counties2['county, state'][i]][1])

In [ ]:
df_counties2['latitude']=lat_series
df_counties2['longitude']=long_series
df_counties2.tail()

### Now we map a heatmap of all the cases over the entire US. This is a different method to show which areas are most dense in virus cases

In [ ]:
most_recent_day2='2020-04-11'
df_counties_curr=df_counties2[df_counties2['date']==most_recent_day2]
df_counties_curr.head()
heatmap_list=df_counties_curr[['latitude','longitude','cases']].values.tolist()

In [ ]:
heatmap_list

In [ ]:
from folium.plugins import HeatMap
base_map=folium.Map(location=[37,-96], zoom_start=4)
HeatMap(data=heatmap_list,radius=10).add_to(base_map)
base_map    

### Finally we visualize the spread of COVID-19 over both TIME AND SPACE. Let's extend our heatmap list in the time dimension so we can watch it's evolution

In [ ]:
heatmap_list_time=[]
for date in df_counties2['date'].unique():
    df_counties_curr=df_counties2[df_counties2['date']==date]
    heatmap_list_time.append(df_counties_curr[['latitude','longitude','cases']].values.tolist())

In [ ]:
from folium.plugins import HeatMapWithTime
base_map=folium.Map(location=[37,-96], zoom_start=4)
HeatMapWithTime(data=heatmap_list_time, radius=10,auto_play=True).add_to(base_map)
base_map

# Results and Discussion

It is clear from the above analysis that the number of coronavirus cases will continue to increase in every state of the US. It is also clear the areas with larger populations have a larger number of cases, which is what would be expected, unless drastic quarantine or stay-at-home orders were issued in those areas.

# Conclusion

Using Python and various libraries for Python I have analyzed and shown visualization of the spread of coronavirus over the US in both space and time. It is clear from the above analysis that the situation is serious, and the spread isn’t stopping anytime soon. 